![header](../figures/DC_MapMed_OSE-banner.png)

***
**Authors:**  Datlas & IGE <br>
**Copyright:** 2022 Datlas & IGE <br>
**License:** MIT

<div class="alert alert-block alert-success">
    <h1><center>Perform baseline OI</center></h1>
    <h5><center>
    <h5><center>This notebook shows how to address the data challenge by performing a mapping technique baseline: an optimal interpolation.</h5> 
</div> 






The notebook is structured as follow: 

    1) Optimal interpolation principle
    2) Set up optimal interpolation parameters,
    3) Read observations,
    4) Run optimal interpolation 
    5) Save the outputs  

In [1]:
%matplotlib inline

import xarray as xr
import numpy
import warnings
import logging
import sys
import os
warnings.filterwarnings('ignore')
sys.path.remove('/Users/sammymetref/Documents/DataChallenges/DC_IW_crossoverCal/2022a_mapping_HFdynamic')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

sys.path.append('..')

from src.mod_oi import *
from src.mod_inout import *

<div class="alert alert-info" role="alert">

<h2> Optimal interpolation principle </h2>

</div>



Here, we assume a vector of observations, noted $y$ defined as:

$$y = H x + \epsilon $$

where $H$ is a linear observation operator between the reconstruction grid space and the observation space
, $x$ is the state to estimate and $\epsilon$ is an independent observation error.

The optimal interpolation consists in estimating an analysed state $x_{a}$ in combining the available observations to approximate the real state $x$:

$$x_{a} = K y $$
where $K$ is the weigth matrix defined as:

$$ K = BH^T(HBH^T + R)^{-1} $$

$B$ is the covariance matrix of $x$, and $R$ the covariance matrix of the error vector $\epsilon$ ($^T$ is the transpose operator)

<div class="alert alert-info" role="alert">

<h2> Set up optimal interpolation parameters </h2>

</div>

In [2]:
# OI Grid
lon_min = 1.                                           # domain min longitude
lon_max = 20.                                           # domain max longitude
lat_min = 30.                                            # domain min latitude
lat_max = 45.                                            # domain max latitude
time_min = numpy.datetime64('2021-01-01')                # domain min time
time_max = numpy.datetime64('2021-03-31')                # domain max time
dx = 0.1                                                 # zonal grid spatial step (in degree)
dy = 0.1                                                 # meridional grid spatial step (in degree)
dt = numpy.timedelta64(1, 'D')                           # temporal grid step

glon = numpy.arange(lon_min, lon_max + dx, dx)           # output OI longitude grid
glat = numpy.arange(lat_min, lat_max + dy, dy)           # output OI latitude grid
gtime = numpy.arange(time_min, time_max + dt, dt)        # output OI time grid

# OI parameters
Lx = 1.                                                  # Zonal decorrelation scale (in degree)
Ly = 1.                                                  # Meridional decorrelation scale (in degree)
Lt = 7.                                                  # Temporal decorrelation scale (in days)
noise = 0.05                                             # Noise level (5%)

<div class="alert alert-info" role="alert">

<h2> Read observations </h2>

</div>

In [3]:
# Download the compressed file containing the obs
!wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/ocean-data-challenges/dc_MapMed_OSE/dc_obs.tar.gz
# Uncompress in ../inputs
!tar -xvf dc_obs.tar.gz --directory ../inputs
# Remove compressed file
!rm -f dc_obs.tar.gz

--2022-09-28 12:03:10--  https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/ocean-data-challenges/dc_MapMed_OSE/dc_obs.tar.gz
Résolution de ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)… 129.88.191.39
Connexion à ige-meom-opendap.univ-grenoble-alpes.fr (ige-meom-opendap.univ-grenoble-alpes.fr)|129.88.191.39|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 200
Taille : 1637056 (1,6M) [application/octet-stream]
Sauvegarde en : « dc_obs.tar.gz »

dc_obs.tar.gz       100%[===================>]   1,56M  --.-KB/s    ds 0,02s   

2022-09-28 12:03:10 (79,9 MB/s) — « dc_obs.tar.gz » sauvegardé [1637056/1637056]

x dc_obs/
x dc_obs/dt_med_alg_phy_l3_20210101-20210331_0-20_30-45.nc
x dc_obs/dt_med_j3_phy_l3_20210101-20210331_0-20_30-45.nc
x dc_obs/dt_med_h2b_phy_l3_20210101-20210331_0-20_30-45.nc
x dc_obs/dt_med_mdt_phy_l4_0.5-20.0_29-45.nc
x dc_obs/dt_med_s3b_phy_l3_20210101-20210331_0-20_30-45.nc
x dc_o

In [4]:
# Define outputs
output_directory = '../results/'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)  
output_oi = f'{output_directory}/dt_med_mapping_baselineOI.nc'

<div class="alert alert-info" role="alert">

<h2> Run optimal interpolation </h2>

</div>

In [5]:
%%time
# set OI param & grid
ds_oi1_param = oi_param(Lx, Ly, Lt, noise)
ds_oi1_grid = oi_grid(glon, glat, gtime)

# Read input obs + discard a bit...
coarsening = {'time': 5}
ds_oi1_obs = read_obs('../inputs/dc_obs/*l3*.nc',ds_oi1_grid,ds_oi1_param,coarsening)
ds_oi1_obs.sla_unfiltered.values = ds_oi1_obs.sla_unfiltered + ds_oi1_obs.mdt

# Run OI (take 1h on my laptop)
for it in range(len(gtime)):
    oi_core(it, ds_oi1_grid, ds_oi1_param, ds_oi1_obs)
    
ds_oi = ds_oi1_grid.drop(['gtime', 'ng', 'glon2', 'glat2', 'fglon', 'fglat', 'nobs'])
ds_oi = ds_oi.rename({'gssh': 'ssh'})

INFO:root:     Set OI params...
INFO:root:     Set OI grid...
INFO:root:     Reading observations...


CPU times: user 1h 5min 25s, sys: 2min 12s, total: 1h 7min 38s
Wall time: 40min 32s


<div class="alert alert-info" role="alert">

<h2> Save the outputs </h2>

</div>

In [6]:
ds_oi.to_netcdf(output_oi)